# Distributed probability map

This notebook contains a prototype for the impact map using the scaler and the pointer:

Basically the idea would be to query the sourcing location data using the clien filters, so for each georegion we will need to do the following:

    distributed impact = (Value *  H3 prod values (pointer))/ Scaler
    



In [1]:
def psql(query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        conn.rollback()
        print(e)

In [2]:
#import libraries
from psycopg2.pool import ThreadedConnectionPool
import pandas as pd


In [3]:
## env file for gcs upload
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value
        
list(env.keys())

['API_SERVICE_PORT',
 'API_POSTGRES_HOST',
 'API_POSTGRES_PORT',
 'API_POSTGRES_USERNAME',
 'API_POSTGRES_PASSWORD',
 'API_POSTGRES_DATABASE',
 'CLIENT_SERVICE_PORT']

In [4]:
# conect to ddbb
postgres_thread_pool = ThreadedConnectionPool(1, 50,
                                              host=env['API_POSTGRES_HOST'],
                                              port=env['API_POSTGRES_PORT'],
                                              user=env['API_POSTGRES_USERNAME'],
                                              password=env['API_POSTGRES_PASSWORD']
                                              )
#get list of sourcing records to iterate:
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()

In [5]:
#filters to apply in the query
indicator_id = 'e2c00251-fe31-4330-8c38-604535d795dc'
material_id = '52370288-8c6c-4f31-919c-d30538d39ee3'##palm oil -- we can select all material or specific ones
origins = 'a02bee9f-5cc6-4dce-99e1-60be757daee7'## admin region Id-- we need to get the paret and chlids to filter the sourcing locations 
#suppliers = 'f634e464-c9ca-469d-85d6-df3cd1877f26' suppliers/producers



In [17]:
## SELECT FIRST THE GEOREGION
SQL_GET_H3_UNCOMPACT_GEO_REGION = """
CREATE OR REPLACE FUNCTION get_h3_uncompact_geo_region(geo_region_id uuid, h3_resolution int)
RETURNS TABLE (h3index h3index) AS 
$$
    SELECT h3_uncompact(geo_region."h3Compact"::h3index[], h3_resolution) h3index
    FROM geo_region WHERE geo_region.id = geo_region_id
$$ 
LANGUAGE SQL;
"""

SQL_GET_H3_MATERIAL_TABLE_COLUMN = """
CREATE OR REPLACE FUNCTION get_h3_material_table_column(h3DataId uuid)
RETURNS TABLE (h3_table_name varchar, h3_column_name varchar) AS
$$
    SELECT h3_data."h3tableName", h3_data."h3columnName"
    FROM h3_data
    WHERE h3_data.id = h3DataId 
    LIMIT 1;
$$
LANGUAGE SQL;
"""



##NOTE1: Update parsing of material h3 table name and column which is now fixed
##NOTE2: Try to return h3index and value
##NOTE3: It seems that the h3DataId in the ddbb is pointing at the indicator id rather than the production tables
##NOTE4:h3DataId tyoe in the indicator record  entity needs to be updated from text to uuid
SQL_GET_H3_DATA_OVER_GEO_REGION = SQL_GET_H3_MATERIAL_TABLE_COLUMN+SQL_GET_H3_UNCOMPACT_GEO_REGION+"""
CREATE OR REPLACE FUNCTION get_h3_data_over_georegion(
    geo_region_id uuid, 
    h3DataId uuid
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar := 'h3_grid_spam2010v2r0_global_prod';
        material_h3_column_name varchar:= 'spam2010V2R0GlobalPBarlA';
        h3_resolution integer := 6;
        value float;

    BEGIN
        -- Get h3data table name and column name for given material
        SELECT * INTO material_h3_table_name, material_h3_column_name
        FROM get_h3_material_table_column(h3DataId);

        -- Sum table column over region
        EXECUTE format(
            'SELECT h3grid.%I
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid ON h3grid.h3index = geo_region.h3index
            WHERE h3grid.%I > 0
            ', material_h3_column_name, material_h3_table_name, material_h3_column_name)
            USING geo_region_id, h3_resolution
            INTO value;
        RETURN value;
    END;
$$
LANGUAGE plpgsql;
"""

In [12]:
pd.read_sql_query(
    SQL_GET_H3_UNCOMPACT_GEO_REGION+
    """
    SELECT ir.value, ir.scaler, ir."h3DataId", sl."geoRegionId" 
    FROM sourcing_location sl 
    RIGHT JOIN sourcing_records sr ON sr."sourcingLocationId" =sl.id 
    INNER JOIN indicator_record ir on ir."sourcingRecordId" =sr.id 
    WHERE sl."adminRegionId" in (
        SELECT ar.id 
        FROM admin_region ar 
        WHERE mpath LIKE '%a02bee9f-5cc6-4dce-99e1-60be757daee7%' -- list of admin regions ids provided by the user
        )
    AND sl."materialId" IN ('52370288-8c6c-4f31-919c-d30538d39ee3')-- list material ids provided by the user
    ---and sl."t1SupplierId" = 'a02bee9f-5cc6-4dce-99e1-60be757daee7' OR sl."producerId" ='a02bee9f-5cc6-4dce-99e1-60be757daee7' -- add supplier filter
    AND ir."indicatorId" ='e2c00251-fe31-4330-8c38-604535d795dc'
    """, conn)

,value,scaler,h3DataId,geoRegionId
0,2.703041e+11,6535618.00,84bdad6c-a318-4c58-989e-42f9328d9411,2b7f0fbb-f525-4619-a7da-f8661a930683
1,2.743863e+11,6535618.00,84bdad6c-a318-4c58-989e-42f9328d9411,2b7f0fbb-f525-4619-a7da-f8661a930683
2,2.784685e+11,6535618.00,84bdad6c-a318-4c58-989e-42f9328d9411,2b7f0fbb-f525-4619-a7da-f8661a930683
3,2.826307e+11,6535618.00,84bdad6c-a318-4c58-989e-42f9328d9411,2b7f0fbb-f525-4619-a7da-f8661a930683
4,2.868730e+11,6535618.00,84bdad6c-a318-4c58-989e-42f9328d9411,2b7f0fbb-f525-4619-a7da-f8661a930683
...,...,...,...,...
2030,2.328857e+10,3661818.25,84bdad6c-a318-4c58-989e-42f9328d9411,92ae69e1-c5ae-4b9e-ab74-b7ab0f6d7017
2031,2.363849e+10,3661818.25,84bdad6c-a318-4c58-989e-42f9328d9411,92ae69e1-c5ae-4b9e-ab74-b7ab0f6d7017
2032,2.399586e+10,3661818.25,84bdad6c-a318-4c58-989e-42f9328d9411,92ae69e1-c5ae-4b9e-ab74-b7ab0f6d7017
2033,2.435323e+10,3661818.25,84bdad6c-a318-4c58-989e-42f9328d9411,92ae69e1-c5ae-4b9e-ab74-b7ab0f6d7017


In [18]:
%%time
#parse data from user filters
df_impact_map = pd.read_sql_query(
    SQL_GET_H3_DATA_OVER_GEO_REGION+\
    SQL_GET_H3_UNCOMPACT_GEO_REGION+
    f"""
    SELECT get_h3_uncompact_geo_region(sl."geoRegionId",6) as v, sum((ir.value * get_h3_data_over_georegion(sl."geoRegionId", ir."h3DataId")) / ir.scaler) as k 
    FROM sourcing_location sl 
    RIGHT JOIN sourcing_records sr ON sr."sourcingLocationId" =sl.id 
    INNER JOIN indicator_record ir on ir."sourcingRecordId" =sr.id 
    WHERE sl."adminRegionId" in (
        SELECT ar.id 
        FROM admin_region ar 
        WHERE mpath LIKE '%a02bee9f-5cc6-4dce-99e1-60be757daee7%' -- list of admin regions ids provided by the user
        )
    AND sl."materialId" IN ('52370288-8c6c-4f31-919c-d30538d39ee3')-- list material ids provided by the user
    ---and sl."t1SupplierId" = 'a02bee9f-5cc6-4dce-99e1-60be757daee7' OR sl."producerId" ='a02bee9f-5cc6-4dce-99e1-60be757daee7' -- add supplier filter
    AND ir."indicatorId" ='e2c00251-fe31-4330-8c38-604535d795dc'
    GROUP BY v
    """, conn)
df_impact_map

CPU times: user 24.7 ms, sys: 8.24 ms, total: 33 ms
Wall time: 5min 36s


,v,k
0,869534db7ffffff,1.218707e+13
1,869534dafffffff,1.218707e+13
2,869534da7ffffff,1.218707e+13
3,869534d9fffffff,1.218707e+13
4,869534d97ffffff,1.218707e+13
...,...,...
16912,866425917ffffff,5.134925e+14
16913,86642590fffffff,5.134925e+14
16914,866425907ffffff,5.134925e+14
16915,86642582fffffff,5.134925e+14


In [20]:
query1= SQL_GET_H3_DATA_OVER_GEO_REGION+\
    SQL_GET_H3_UNCOMPACT_GEO_REGION+\
    f"""
    SELECT get_h3_uncompact_geo_region(sl."geoRegionId",6) as v, sum((ir.value * get_h3_data_over_georegion(sl."geoRegionId", ir."h3DataId")) / ir.scaler) as k 
    FROM sourcing_location sl 
    RIGHT JOIN sourcing_records sr ON sr."sourcingLocationId" =sl.id 
    INNER JOIN indicator_record ir on ir."sourcingRecordId" =sr.id 
    WHERE sl."adminRegionId" in (
        SELECT ar.id 
        FROM admin_region ar 
        WHERE mpath LIKE '%a02bee9f-5cc6-4dce-99e1-60be757daee7%' -- list of admin regions ids provided by the user
        )
    AND sl."materialId" IN ('52370288-8c6c-4f31-919c-d30538d39ee3')-- list material ids provided by the user
    ---and sl."t1SupplierId" = 'a02bee9f-5cc6-4dce-99e1-60be757daee7' OR sl."producerId" ='a02bee9f-5cc6-4dce-99e1-60be757daee7' -- add supplier filter
    AND ir."indicatorId" ='e2c00251-fe31-4330-8c38-604535d795dc'
    GROUP BY v
    """
%timeit psql(query1)

5min 17s ± 15.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
print(psql(query1))

[('869534db7ffffff', 12187071255743.525), ('869534dafffffff', 12187071255743.525), ('869534da7ffffff', 12187071255743.525), ('869534d9fffffff', 12187071255743.525), ('869534d97ffffff', 12187071255743.525), ('869534d8fffffff', 12187071255743.525), ('869534d87ffffff', 12187071255743.527), ('869534d77ffffff', 21909239241972.605), ('869534d6fffffff', 21909239241972.605), ('869534d67ffffff', 21909239241972.6), ('869534d5fffffff', 12187071255743.523), ('869534d57ffffff', 12187071255743.523), ('869534d4fffffff', 20795114656815.67), ('869534d47ffffff', 20795114656815.664), ('869534d37ffffff', 12187071255743.523), ('869534d2fffffff', 21909239241972.605), ('869534d27ffffff', 21909239241972.605), ('869534d1fffffff', 12187071255743.525), ('869534d17ffffff', 12187071255743.525), ('869534d0fffffff', 12187071255743.523), ('869534d07ffffff', 12187071255743.525), ('869534ca7ffffff', 12187071255743.527), ('869534c37ffffff', 12187071255743.525), ('869534c2fffffff', 12187071255743.523), ('869534c27ffffff'